In [1]:
import os

import numpy as np
import math
import time
from tqdm import tqdm #time module progress bar
import matplotlib.pyplot as plt
from random import shuffle
import torch
from torch.utils.data import Dataset, DataLoader
import shutil 
import json
from bm3d import bm3d
import matplotlib.pyplot as plt
from bm3d import bm3d, BM3DProfile
import multiprocessing as mp
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compute_ssim

In [ ]:
Pre-Processing.ipynb
data/
    dataset/
        Free/
            IXI-PD
                training
                    (PD images)
                validation
                    (PD images)
                testing
                    (PD images)
                
            IXI-T2/
                training
                    (T2 images)
                validation
                    (T2 images)
                testing
                    (T2 images)
        noise_{noise_level}/
            testing
                IXI-PD/
                IXI-T2/
            validation
                IXI-PD/
                IXI-T2/
            training
                IXI-PD/
                IXI-T2/
    patchs32_32_{noise_level}/
        free/
            testing
                IXI-PD/
                IXI-T2/
            validation
                IXI-PD/
                IXI-T2/
            training
IXI-PD/
                IXI-T2/
        noised/
            testing
                IXI-PD/
                IXI-T2/
            validation
                IXI-PD/
                IXI-T2/
            training
                IXI-PD/
                IXI-T2/
    mixdata/
        free/
            IXI-PD
                testing
                    (PD images)
                validation
                    (PD images)
                training
                    (PD images)
            IXI-T2/
                testing
                    (T2 images)
validation
                    (T2 images)
                training
                    (T2 images)
        noised/
            IXI-PD
                testing
                    (PD images)
                validation
                    (PD images)
                training
                    (PD images)
            IXI-T2/
                testing
                    (T2 images)
                validation
                    (T2 images)
                training
                    (T2 images)
    denoised_images/
        patchs32_32_{noise_level}/
            noised/
                IXI-PD/
                    denoised/
        patchs32_32_{noise_level}/
            noised/
                IXI-PD/
                    denoised/
        patchs32_32_{noise_level}/
            noised/
                IXI-PD/
                    denoised/
        patchs32_32_{noise_level}/
            noised/
                IXI-PD/
                    denoised/
    reconstructed_images
        patchs32_32_{noise_level}/
            noised/
                IXI-PD/
                    reconstructed/
            free/
        patchs32_32_{noise_level}/
            noised/
                IXI-PD/
                    reconstructed/
            free/
                IXI-PD/
                    reconstructed/
        patchs32_32_{noise_level}/
            noised/
                IXI-PD/
                    reconstructed/
            free/
                IXI-PD/
                    reconstructed/
        patchs32_32_{noise_level}/
            noised/
                IXI-PD/
                    reconstructed/
            free/
                IXI-PD/
                    reconstructed/

In [ ]:
## Please set your working directory to C:\Users\YourUsername\Desktop\data

In [29]:
## Recontructing the images from the patches does in preprocessing

In [2]:

def reconstruct_image_from_patches(patch_file):
    #From Preprocessing Code used to create patches
    patch_size=32
    stride=8

    patches = np.load(patch_file)  
    patches = np.squeeze(patches)  # get rid of channel 1
    patches = patches.transpose(0, 2, 3, 1)  # Shape: (num, 32, 32, 6)
    num = patches.shape[0]
    row = int(np.sqrt(num))
    col = row
    reconstructed_height = (row - 1) * stride + patch_size
    reconstructed_width = (col - 1) * stride + patch_size
    depth = 6

    row = int(np.sqrt(num))
    col = row
    reconstructed_height = (row - 1) * stride + patch_size
    reconstructed_width = (col - 1) * stride + patch_size
    
    reconstructed_image = np.zeros((reconstructed_height, reconstructed_width, depth), dtype=np.float32) #initalize matrix for image with zeros
    weight_matrix = np.zeros((reconstructed_height, reconstructed_width, depth), dtype=np.float32)

    
    index = 0
    for y in range(0, reconstructed_height - patch_size + 1, stride): #going through height
        for x in range(0, reconstructed_width - patch_size + 1, stride): #going through width of image
            if index >= num: #end when number of patches
                break #done before the rest to avoid extra iterations
            patch = patches[index] #picks out patch to add to image matrix
            reconstructed_image[y:y+patch_size, x:x+patch_size, :] += patch #adds patch to image
            weight_matrix[y:y+patch_size, x:x+patch_size, :] += 1 #used to deal with the inherent overlapping of the patches
            index += 1

    # Normalize overlapping 
    weight_matrix[weight_matrix == 0] = 1
    reconstructed_image /= weight_matrix

    return reconstructed_image



In [3]:
def reconstruct_all_images(base_dir, output_base_dir):
    noise_levels = [1, 7, 13] 
    data_types = ['free', 'noised']
    modality = 'IXI-PD'
    dataset_split = 'testing'

    for x in noise_levels:
        
        patches_dir = os.path.join(base_dir, f'patchs32_32_{x}')
        print(f"Patches directory: {os.path.abspath(patches_dir)}")
        for a in data_types:
            
            data_dir = os.path.join(patches_dir, a, modality, dataset_split)
            print(f"Data directory: {os.path.abspath(data_dir)}")
            if not os.path.exists(data_dir):
                print(f"Directory does not exist: {os.path.abspath(data_dir)}")
                continue
            
            patch_files = [f for f in os.listdir(data_dir) if f.endswith('.npy')] #getting all patch files
            
            for patch_file in tqdm(patch_files, desc=f"Processing {a} data at noise level {x}"):
              
                patch_file_path = os.path.join(data_dir, patch_file)
        
                reconstructed_image = reconstruct_image_from_patches(patch_file_path) #Using patch reconstruction image from above
                
                output_dir = os.path.join(output_base_dir, f'patchs32_32_{x}',a, modality,'reconstructed')
              
                os.makedirs(output_dir, exist_ok=True)
                
                output_file_path = os.path.join(output_dir, patch_file.replace('.npy', '_reconstructed.npy'))
                
                np.save(output_file_path, reconstructed_image)

In [4]:

if __name__ == '__main__':
    # Set base_dir to the current working directory
    base_dir = os.getcwd()

    # Set output_base_dir relative to base_dir
    output_base_dir = os.path.join(base_dir, 'reconstructed_images')

    # Print the paths to verify
    print(f"Current working directory: {base_dir}")
    print(f"Base directory (absolute path): {os.path.abspath(base_dir)}")
    print(f"Output directory (absolute path): {os.path.abspath(output_base_dir)}")

    reconstruct_all_images(base_dir, output_base_dir)

Current working directory: C:\Users\Regina\Desktop\Progress Report 2
Base directory (absolute path): C:\Users\Regina\Desktop\Progress Report 2
Output directory (absolute path): C:\Users\Regina\Desktop\Progress Report 2\reconstructed_images
Patches directory: C:\Users\Regina\Desktop\Progress Report 2\patchs32_32_1
Data directory: C:\Users\Regina\Desktop\Progress Report 2\patchs32_32_1\free\IXI-PD\testing


Processing free data at noise level 1: 100%|██████████| 10/10 [00:01<00:00,  8.50it/s]


Data directory: C:\Users\Regina\Desktop\Progress Report 2\patchs32_32_1\noised\IXI-PD\testing


Processing noised data at noise level 1: 100%|██████████| 10/10 [00:01<00:00,  8.14it/s]


Patches directory: C:\Users\Regina\Desktop\Progress Report 2\patchs32_32_7
Data directory: C:\Users\Regina\Desktop\Progress Report 2\patchs32_32_7\free\IXI-PD\testing


...

Processing noised data at noise level 7: 100%|██████████| 10/10 [00:01<00:00,  8.03it/s]


Patches directory: C:\Users\Regina\Desktop\Progress Report 2\patchs32_32_13
Data directory: C:\Users\Regina\Desktop\Progress Report 2\patchs32_32_13\free\IXI-PD\testing


Processing free data at noise level 13: 100%|██████████| 10/10 [00:01<00:00,  9.69it/s]


Data directory: C:\Users\Regina\Desktop\Progress Report 2\patchs32_32_13\noised\IXI-PD\testing


Processing noised data at noise level 13: 100%|██████████| 10/10 [00:01<00:00,  8.90it/s]


In [ ]:
## BM3D on noised MRIs

In [5]:
def denoise_all_reconstructed_images(reconstructed_base_dir, denoised_base_dir):
    noise_levels = [1, 7, 13]  
    types = 'noised'  
    modality = 'IXI-PD' #the preprocessing code includes IXI-T2 images but to cut down data we are only working with IXI-T2
    

    for x in noise_levels:
        
        patches_dir = os.path.join(reconstructed_base_dir, f'patchs32_32_{x}')
        
        reconstructed_dir = os.path.join(patches_dir,types,modality,'reconstructed')
   
        
       
        reconstructed_files = [f for f in os.listdir(reconstructed_dir) if f.endswith('_reconstructed.npy')]
        
        for reconstructed_file in tqdm(reconstructed_files, desc=f"Processing noise level {x}"):
            reconstructed_file_path = os.path.join(reconstructed_dir, reconstructed_file)
            reconstructed_image = np.load(reconstructed_file_path)
            
            # normalizing (not sure if needed but done in case
            reconstructed_image = reconstructed_image.astype(np.float32)
            reconstructed_image -= reconstructed_image.min()
            reconstructed_image /= reconstructed_image.max()
       
            
            
            #this is an estimate of the sigma_psd
            sigma_psd =x/100.0
            
            
            # BM3D
            denoised_slices = [] #initalize
            for i in range(reconstructed_image.shape[2]): #accessing slices
                slice = reconstructed_image[:, :, i]
                denoised_slice = bm3d(slice, sigma_psd)
                denoised_slices.append(denoised_slice)
            denoised_image = np.stack(denoised_slices, axis=-1)
            
            
            denoised_dir = os.path.join(denoised_base_dir,f'patchs32_32_{x}',types,modality,'denoised')
            os.makedirs(denoised_dir, exist_ok=True)
            
            name = reconstructed_file.replace('_reconstructed.npy', '_denoised.npy')
            denoised_file_path = os.path.join(denoised_dir, name)
            np.save(denoised_file_path, denoised_image)
            
 
            print(f"Denoised image saved to {denoised_file_path}")



In [6]:
if __name__ == '__main__':
   
    current_dir = os.getcwd()

    
    reconstructed_base_dir = os.path.join(current_dir, 'reconstructed_images')

   
    denoised_base_dir = os.path.join(current_dir, 'denoised_images')

 
    denoise_all_reconstructed_images(reconstructed_base_dir, denoised_base_dir)

Processing noise level 1:  10%|█         | 1/10 [00:16<02:24, 16.07s/it]

Denoised image saved to C:\Users\Regina\Desktop\Progress Report 2\denoised_images\patchs32_32_1\noised\IXI-PD\denoised\IXI160-HH-1637-PD-1_denoised.npy


Processing noise level 1:  20%|██        | 2/10 [00:30<02:02, 15.31s/it]

Denoised image saved to C:\Users\Regina\Desktop\Progress Report 2\denoised_images\patchs32_32_1\noised\IXI-PD\denoised\IXI161-HH-2533-PD-1_denoised.npy


Processing noise level 1:  30%|███       | 3/10 [00:46<01:48, 15.53s/it]

...

Denoised image saved to C:\Users\Regina\Desktop\Progress Report 2\denoised_images\patchs32_32_13\noised\IXI-PD\denoised\IXI174-HH-1571-PD-13_denoised.npy


Processing noise level 13:  90%|█████████ | 9/10 [02:26<00:16, 16.10s/it]

Denoised image saved to C:\Users\Regina\Desktop\Progress Report 2\denoised_images\patchs32_32_13\noised\IXI-PD\denoised\IXI175-HH-1570-PD-13_denoised.npy


Processing noise level 13: 100%|██████████| 10/10 [02:49<00:00, 17.00s/it]

Denoised image saved to C:\Users\Regina\Desktop\Progress Report 2\denoised_images\patchs32_32_13\noised\IXI-PD\denoised\IXI176-HH-1604-PD-13_denoised.npy


In [ ]:
## Getting PSNR values

In [9]:
def compute_psnr(truth, target):
    #normalizing the free images
    truth = truth.astype(np.float32)
    target = target.astype(np.float32)
    truth -= truth.min()
    truth /= truth.max() if truth.max() != 0 else 1
    target -= target.min()
    target /= target.max() if target.max() != 0 else 1
    
    # Compute PSNR for each slice
    psnr_values = []
    num_slices = truth.shape[2]
    for i in range(num_slices):
        ref_slice = truth[:, :, i]
        target_slice = target[:, :, i]
        psnr = compare_psnr(ref_slice, target_slice, data_range=1.0)
        psnr_values.append(psnr)
    return np.mean(psnr_values)  # Average PSNR over slices

def main():
    base_dir = os.getcwd()
    noise_levels = [1, 7, 13]
    modality = 'IXI-PD'
    averagepsnr_noised = {}
    average_psnr_denoised = {}

    for x in noise_levels:
        
        reconstructed_free_dir = os.path.join(base_dir,'reconstructed_images',f'patchs32_32_{x}','free',modality,'reconstructed')
        
        reconstructed_noised_dir = os.path.join(base_dir,'reconstructed_images',f'patchs32_32_{x}','noised',modality,'reconstructed')
       
        denoised_dir = os.path.join(base_dir,'denoised_images', f'patchs32_32_{x}','noised',modality,'denoised')

        reconstructed_free_files = [f for f in os.listdir(reconstructed_free_dir) if f.endswith('_reconstructed.npy')]
        psnr_noised_values = []
        psnr_denoised_values = []

        for free_file in tqdm(reconstructed_free_files, desc=f"Processing noise level {x}"):
            base_filename = free_file.replace('_reconstructed.npy', '')
            free_image_path = os.path.join(reconstructed_free_dir, free_file)
            noised_image_name = base_filename + '_reconstructed.npy'
            noised_image_path = os.path.join(reconstructed_noised_dir, noised_image_name)
            denoised_image_name = base_filename + '_denoised.npy'
            denoised_image_path = os.path.join(denoised_dir, denoised_image_name)
            
            free_image = np.load(free_image_path)
            noised_image = np.load(noised_image_path)
            denoised_image = np.load(denoised_image_path)
            
            # Compute PSNR noised
            psnr_noised = compute_psnr(free_image, noised_image)
            psnr_noised_values.append(psnr_noised)
            
            # Compute PSNR denoised
            psnr_denoised = compute_psnr(free_image, denoised_image)
            psnr_denoised_values.append(psnr_denoised)
        
        # PSNRs per noise level
        if psnr_noised_values:
            averagepsnr_noised_level = np.mean(psnr_noised_values)
            averagepsnr_noised[x] = averagepsnr_noised_level
        else:
            averagepsnr_noised[x] = None

        if psnr_denoised_values:
            average_psnr_denoised_level = np.mean(psnr_denoised_values)
            average_psnr_denoised[x] = average_psnr_denoised_level
        else:
            average_psnr_denoised[x] = None

   
    print("\nAverage PSNRs for each noise level:")
    print("Noise Level\tNoised PSNR\tDenoised PSNR\tImprovement")
    for x in sorted(averagepsnr_noised.keys()):
        psnr_noised = averagepsnr_noised[x]
        psnr_denoised = average_psnr_denoised[x]
        if psnr_noised is not None and psnr_denoised is not None:
            improvement = psnr_denoised - psnr_noised
            print(f"{x}\t\t{psnr_noised:.2f} dB\t{psnr_denoised:.2f} dB\t{improvement:.2f} dB")

In [10]:

if __name__ == '__main__':
    main()


Processing noise level 13: 100%|██████████| 10/10 [00:00<00:00, 22.86it/s]


Average PSNRs for each noise level:
Noise Level	Noised PSNR	Denoised PSNR	Improvement
1		38.30 dB	40.37 dB	2.07 dB
7		21.71 dB	25.97 dB	4.26 dB
13		17.13 dB	24.72 dB	7.59 dB


In [13]:
def compute_ssim_metric(truth, target):
    
    truth = truth.astype(np.float32)
    target = target.astype(np.float32)
    truth -= truth.min()
    truth /= truth.max() if truth.max() != 0 else 1
    
    target -= target.min()
    target /= target.max() if target.max() != 0 else 1
    
    
    ssim_values = []
    num_slices = truth.shape[2]
    for i in range(num_slices):
        ref_slice = truth[:, :, i]
        
        target_slice = target[:, :, i]
        ssim, _ = compute_ssim(ref_slice, target_slice, data_range=1.0, full=True)
        ssim_values.append(ssim)
        
    return np.mean(ssim_values)



def main():
    base_dir = os.getcwd()
    noise_levels = [1, 7, 13]
    modality = 'IXI-PD'
    average_ssim_noised = {}
    average_ssim_denoised = {}

    for x in noise_levels:
        
        reconstructed_free_dir = os.path.join(base_dir,'reconstructed_images',f'patchs32_32_{x}','free',modality,'reconstructed')
        
        reconstructed_noised_dir = os.path.join(base_dir,'reconstructed_images',f'patchs32_32_{x}','noised',modality,'reconstructed')
       
        denoised_dir = os.path.join(base_dir,'denoised_images', f'patchs32_32_{x}','noised',modality,'denoised')

        reconstructed_free_files = [f for f in os.listdir(reconstructed_free_dir) if f.endswith('_reconstructed.npy')]
        ssim_noised_values = []
        ssim_denoised_values = []

        for free_file in tqdm(reconstructed_free_files, desc=f"Processing noise level {x}"):
            base_filename = free_file.replace('_reconstructed.npy', '')
            free_image_path = os.path.join(reconstructed_free_dir, free_file)
            noised_image_name = base_filename + '_reconstructed.npy'
            noised_image_path = os.path.join(reconstructed_noised_dir, noised_image_name)
            denoised_image_name = base_filename + '_denoised.npy'
            denoised_image_path = os.path.join(denoised_dir, denoised_image_name)
            
            free_image = np.load(free_image_path)
            noised_image = np.load(noised_image_path)
            denoised_image = np.load(denoised_image_path)
            
            
            ssim_noised = compute_ssim_metric(free_image, noised_image)
            ssim_noised_values.append(ssim_noised)
            
           
            ssim_denoised = compute_ssim_metric(free_image, denoised_image)
            ssim_denoised_values.append(ssim_denoised)
        
        
        if ssim_noised_values:
            average_ssim_noised_level = np.mean(ssim_noised_values)
            average_ssim_noised[x] = average_ssim_noised_level
        else:
            average_ssim_noised[x] = None

        if ssim_denoised_values:
            average_ssim_denoised_level = np.mean(ssim_denoised_values)
            average_ssim_denoised[x] = average_ssim_denoised_level
        else:
            average_ssim_denoised[x] = None

   
    print("\nAverage SSIMs for each noise level:")
    print("Noise Level\tNoised SSIM\tDenoised SSIM\tImprovement")
    for x in sorted(average_ssim_noised.keys()):
        ssim_noised = average_ssim_noised[x]
        ssim_denoised = average_ssim_denoised[x]
        if ssim_noised is not None and ssim_denoised is not None:
            improvement = ssim_denoised - ssim_noised
            print(f"{x}\t\t{ssim_noised:.4f}\t\t{ssim_denoised:.4f}\t\t{improvement:.4f}")


In [14]:

if __name__ == '__main__':
    main()


Processing noise level 13: 100%|██████████| 10/10 [00:00<00:00, 15.87it/s]


Average SSIMs for each noise level:
Noise Level	Noised SSIM	Denoised SSIM	Improvement
1		0.7845		0.8211		0.0366
7		0.3344		0.5070		0.1727
13		0.2166		0.4595		0.2429
